In [1]:
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import json

In [5]:
# f = open()
data_df = pd.read_csv('src/api/data/cumulative-deaths-and-cases-covid-19.csv')

In [7]:
grouped = data_df.groupby('Entity')

In [10]:
data_df['Entity'].unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'American Samoa',
       'Andorra', 'Angola', 'Anguilla', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Aruba', 'Asia', 'Asia excl. China',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bonaire Sint Eustatius and Saba',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo', 'Cook Islands',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor',
       'Ecuador', 'Egypt', 'El Salvador', 'Equator

In [11]:
data_df = data_df[data_df["Entity"] == 'Austria']

In [12]:
data_df

,Entity,Code,Day,Total confirmed deaths due to COVID-19,Total confirmed cases of COVID-19
22457,Austria,AUT,2020-03-15,1.0,588.0
22458,Austria,AUT,2020-03-16,1.0,588.0
22459,Austria,AUT,2020-03-17,1.0,588.0
22460,Austria,AUT,2020-03-18,1.0,588.0
22461,Austria,AUT,2020-03-19,1.0,588.0
...,...,...,...,...,...
23986,Austria,AUT,2020-03-10,NaN,75.0
23987,Austria,AUT,2020-03-11,NaN,75.0
23988,Austria,AUT,2020-03-12,NaN,75.0
23989,Austria,AUT,2020-03-13,NaN,75.0


In [13]:
data_df.isna().sum()

Entity                                     0
Code                                       0
Day                                        0
Total confirmed deaths due to COVID-19    14
Total confirmed cases of COVID-19          0
dtype: int64

In [14]:
data_df.index = pd.DatetimeIndex(pd.to_datetime(data_df["Day"])) #.sort_index(ascending=True, inplace=False)


In [15]:
data_df

,Entity,Code,Day,Total confirmed deaths due to COVID-19,Total confirmed cases of COVID-19
Day,,,,,
2020-03-15,Austria,AUT,2020-03-15,1.0,588.0
2020-03-16,Austria,AUT,2020-03-16,1.0,588.0
2020-03-17,Austria,AUT,2020-03-17,1.0,588.0
2020-03-18,Austria,AUT,2020-03-18,1.0,588.0
2020-03-19,Austria,AUT,2020-03-19,1.0,588.0
...,...,...,...,...,...
2020-03-10,Austria,AUT,2020-03-10,NaN,75.0
2020-03-11,Austria,AUT,2020-03-11,NaN,75.0
2020-03-12,Austria,AUT,2020-03-12,NaN,75.0


In [16]:
my_range = pd.date_range(start=data_df['Day'].min(), end=data_df['Day'].max(), freq='D')


In [17]:
len(my_range[~my_range.isin(data_df.index)])


0

In [18]:
my_range.drop_duplicates()

DatetimeIndex(['2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04',
               '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08',
               '2020-03-09', '2020-03-10',
               ...
               '2024-05-03', '2024-05-04', '2024-05-05', '2024-05-06',
               '2024-05-07', '2024-05-08', '2024-05-09', '2024-05-10',
               '2024-05-11', '2024-05-12'],
              dtype='datetime64[ns]', length=1534, freq='D')

In [19]:
print(data_df.shape)
new_df = pd.DataFrame()
new_df.index = my_range
new_df["Day"] = my_range
data_df = pd.merge(data_df, new_df, how="outer", right_index=True, left_index=True, validate="many_to_many")

(1534, 5)


In [20]:
data_df.index.drop_duplicates()
print(data_df)


             Entity Code       Day_x  Total confirmed deaths due to COVID-19  \
2020-03-01  Austria  AUT  2020-03-01                                     NaN   
2020-03-02  Austria  AUT  2020-03-02                                     NaN   
2020-03-03  Austria  AUT  2020-03-03                                     NaN   
2020-03-04  Austria  AUT  2020-03-04                                     NaN   
2020-03-05  Austria  AUT  2020-03-05                                     NaN   
...             ...  ...         ...                                     ...   
2024-05-08  Austria  AUT  2024-05-08                                 22534.0   
2024-05-09  Austria  AUT  2024-05-09                                 22534.0   
2024-05-10  Austria  AUT  2024-05-10                                 22534.0   
2024-05-11  Austria  AUT  2024-05-11                                 22534.0   
2024-05-12  Austria  AUT  2024-05-12                                 22534.0   

            Total confirmed cases of CO

In [21]:
data_df = data_df.groupby(data_df.index).last()

In [22]:
data_df = data_df.drop_duplicates(keep='last')

In [23]:
print(f"Duplicates: {data_df.index.duplicated()}")


Duplicates: [False False False ... False False False]


In [24]:
data_df = data_df.drop(columns=['Day_x', 'Day_y'])


In [25]:
data_df["Day"] = data_df.index

In [26]:
data_df.isna().sum()

Entity                                     0
Code                                       0
Total confirmed deaths due to COVID-19    14
Total confirmed cases of COVID-19          0
Day                                        0
dtype: int64

In [27]:
data_df = data_df.fillna(0)

In [28]:
data_df.to_json('./src/api/data/cumulative-deaths-and-cases-covid_30-05-24_Austria.json', orient="records")